# AUTONORMALIZE DEMO 
Using Autonormalize to normalize a kaggle dataset about food purchasing.

In [1]:
import os

import pandas as pd
import autonormalize as an

In [2]:
food_df = pd.read_csv(os.path.join(os.getcwd(), 'autonormalize/downloads/food.csv'), encoding='latin1')
food_df = food_df.drop(columns=food_df.columns[10:])
print("Rows: "+ str(food_df.shape[0]))
print("Columns: " + str(food_df.shape[1]))
food_df.head(3)

Rows: 21477
Columns: 10


,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71


This dataset has 21477 rows and we've cut it down to 10 columns. As you can see, there are many data dependencies between that columns that obviously should be split up. For example, Area, Area Code, and Area Abreviation obviously should be dependent on each other.

In [3]:
deps_exact = an.find_dependencies(food_df, 1.00)
deps_approx = an.find_dependencies(food_df, 0.96)
print("\nExact dependencies...")
print(deps_exact)
print("\nApproximate dependencies...")
print(deps_approx)


100%|██████████| 10/10 [00:02<00:00,  3.40it/s]


Exact dependencies...
 {Area}  {Area Code}  --> Area Abbreviation
 {Area}  --> Area Code
 {Area Code}  --> Area
 --> Item Code
 --> Item
 {Element}  --> Element Code
 {Element Code}  --> Element
 {Item}  {Area Code}  {Area}  {longitude}  {Element Code}  {latitude}  {Item Code}  {Element}  {Area Abbreviation}  --> Unit
 {Area}  {Area Code}  --> latitude
 {Area}  {Area Code}  --> longitude

Approximate dependencies...
 {Area}  {Area Code}  --> Area Abbreviation
 {Area}  {Area Abbreviation}  --> Area Code
 {Area Abbreviation}  {Area Code}  --> Area
 --> Item Code
 --> Item
 {Element}  --> Element Code
 {Element Code}  --> Element
 {Item}  {Area Code}  {Area}  {longitude}  {Element Code}  {latitude}  {Item Code}  {Element}  {Area Abbreviation}  --> Unit
 {Area}  {Area Abbreviation}  {Area Code}  --> latitude
 {Area}  {Area Abbreviation}  {Area Code}  --> longitude


In [4]:
groupings = an.normalize_dependencies(deps_approx)
for grp in groupings:
    print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~\n')
    print(grp)


~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Area
 --> Item Code
 --> Item
 {Element}  --> Element Code
 {Element Code}  --> Element

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 {Area Abbreviation}  {Area Code}  --> Area
 {Area}  {Area Code}  --> Area Abbreviation
 {Area}  {Area Abbreviation}  --> Area Code
 {Area}  {Area Abbreviation}  {Area Code}  --> latitude
 {Area}  {Area Abbreviation}  {Area Code}  --> longitude

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> longitude
 {longitude}  --> Unit


In [7]:
groupings[1].find_candidate_keys()

[{'Area Abbreviation'}, {'Area Code'}, {'Area'}]

In [8]:
new_dfs = an.split_dataframe(food_df, groupings)

In [9]:
new_dfs

[              Area  Item Code                          Item  Element Code  \
 0      Afghanistan       2511            Wheat and products          5142   
 1      Afghanistan       2805      Rice (Milled Equivalent)          5142   
 2      Afghanistan       2513           Barley and products          5521   
 3      Afghanistan       2513           Barley and products          5142   
 4      Afghanistan       2514            Maize and products          5521   
 5      Afghanistan       2514            Maize and products          5142   
 6      Afghanistan       2517           Millet and products          5142   
 7      Afghanistan       2520                Cereals, Other          5142   
 8      Afghanistan       2531         Potatoes and products          5142   
 9      Afghanistan       2536                    Sugar cane          5521   
 10     Afghanistan       2537                    Sugar beet          5521   
 11     Afghanistan       2542        Sugar (Raw Equivalent)    